# RWKV v5 Wavnet C
This model is based on the RWKV standard 1B5 model

- 24 layers
- 2048 embedding size

Going through the same memory training process as TokenShift, but using the v5 code

See `./notes.md` for how the init model was initilaized.

**Note:** This project assumes you have the rwkv-infctx conda env setup

---

```bash
# ninja-build is required for the new trainer
sudo apt-get install ninja-build

# Update conda & its package listings
# Conda install script can be found here : 
# https://docs.anaconda.com/free/anaconda/install/linux/#installation
conda update conda

# Virtual env, with python 3.10
# python 3.11 have issues with torch.compile / h100s
# and if you want to use 3.11, you will need to do a nightly build install
conda create -n rwkv-infctx python=3.11 pip
conda activate rwkv-infctx

# Install pytorch (>=2.0.1)
conda install -y pytorch==2.0.1 torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

# Verify your pytorch version 
python -c "import torch; print(torch.__version__)"

# We use python -m pip, instead of pip directly, as it resolve issues with venv not loading the right pip
python -m pip install datasets transformers 
python -m pip install lightning==2.0.4 deepspeed==0.9.5
python -m pip install ninja numexpr jsonargparse 'jsonargparse[signatures]'
python -m pip install lm-dataformat ftfy sentencepiece tokenizers wandb
```
---

# Basic Setup

In [1]:
# First lets setup the various directories, and init the model
!mkdir -p ../../../../model/
!mkdir -p ../../../../datapath/
!mkdir -p ../../../../checkpoint/

# Init the model
!cd ../../../../RWKV-v5altwavenet && python3 ./init_model.py --n_layer 24 --n_embd 2048 --vocab_size neox --skip-if-exists ../model/L24-D2048-neox-v5altwavenet-init.pth

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
---- Initializing model ----
No of layers: 24
Embedding size: 2048
Output model path: ../model/L24-D2048-neox-v5altwavenet-init.pth
Vocab size: 50277
---- ----- ----
50277 2048  -0.0001 emb.weight
2048  2048  1.0  blocks.0.att.receptance.weight
2048  2048  1.0  blocks.0.att.key.weight
2048  2048  1.0  blocks.0.att.value.weight
2048  2048  0    blocks.0.att.output.weight
8192  2048  1.0  blocks.0.ffn.key.weight
2048  2048  0    blocks.0.ffn.receptance.weight
2048  8192  0    blocks.0.ffn.value.weight
2048  2048  1.0  blocks.1.att.receptance.weight
2048  2048  1.0  blocks.1.att.key.weight
2048  2048  1.0  blocks.1.att.value.weight
2048  2048  0    blocks.1.att.output.weight
8192  2048  1.0  blocks.1.ffn.key.weight
2048  2048  0    blocks.1.ffn.receptance.weight
2048  8192  0    blocks.1.ffn.value.weight
2048  2048  1.0  blocks.2.att.receptance.weight
2048  2048  1.0  bl

In [2]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="AltWaveV5-C"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5altwavenet/"))
INFERENCE_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5altwavenet/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("INFERENCE_DIR:", INFERENCE_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /root/rwkv-x-playground/notebook/experiment/tokenshift-exp/AltWaveV5-C
INFERENCE_DIR: /root/rwkv-x-playground/RWKV-v5altwavenet
TRAINER_DIR: /root/rwkv-x-playground/RWKV-v5altwavenet
PROJECT_DIR: /root/rwkv-x-playground


## Stage 1 : Foundation model training

In [3]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/AltWaveV5-C-enwiki.yaml"

Found cached dataset parquet (/root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  3.67it/s]
Loading cached processed dataset at /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-f7b060a194034cc1_*_of_00064.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-5130e0b5cfea4510_*_of_00064.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-8cf337c8a178675e_*_of_00064.arrow
Loading cached split indices for

In [4]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/AltWaveV5-C-enwiki.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki Foundation (ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/AltWaveV5-C/AltWaveV5-C-enwiki.yaml', '--trainer.logger.init_args.name=AltWaveV5-C - Enwiki Foundation (ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/AltWaveV5-C/AltWaveV5-C-enwiki.yaml', '--trainer.logger.init_args.name=AltWaveV5-C - Enwiki Foundation (ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto'].
  rank_zero_warn(
/usr/l

In [5]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/AltWaveV5-C-enwiki/last.ckpt" "../model/AltWaveV5-C-Stage1.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/AltWaveV5-C-Stage1.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/AltWaveV5-C-enwiki/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/AltWaveV5-C-Stage1.pth
-rw-r--r-- 1 root root 5.7G Aug  8 15:27 ../model/AltWaveV5-C-Stage1.pth


In [6]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && python3 dragon_test.py ../model/AltWaveV5-C-Stage1.pth "cuda fp32"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
--- DRAGON PROMPT ---
In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese. The researchers had tried to reconstruct this by searching for more and more deer, but discovered that these seals had been covered by the Danish cougars, and that the possibility that they were unlikely to reach a common ancestor could have been more of a threat than the king's hunting of cattle. However, a more recent scenario of this explanation is presented in the German historian and researcher Ernst Wolfgang von Hölling, who was in Berlin at the time the prefect of the Norwegian crown prince.

The Ráklind Kordalind Kordalind Kordalind proxy was among the earliest known searches of the Thracian Kordalind Kordalind Kordalind Kordal

In [7]:
# # Lets do a quick memory test
# # (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
# !python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/AltWaveV5-C-Stage1.pth"

# Stage 2 : Instruct Tuning

In [8]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/AltWaveV5-C-instruct.yaml"

Found cached dataset parquet (/root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 541.48it/s]
Loading cached processed dataset at /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-e4df40d582f09838_*_of_00064.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-6d5405ad1f265e84_*_of_00064.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a236

In [9]:
# Start the instruct finetuning
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/AltWaveV5-C-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Instruct (train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/AltWaveV5-C/AltWaveV5-C-instruct.yaml', '--trainer.logger.init_args.name=AltWaveV5-C - Instruct (train-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/tokenshift-exp/AltWaveV5-C/AltWaveV5-C-instruct.yaml', '--trainer.logger.init_args.name=AltWaveV5-C - Instruct (train-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto'].
  rank_zero_warn(
/usr/loc

In [10]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/AltWaveV5-C-instruct/last.ckpt" "../model/AltWaveV5-C-Stage2.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/AltWaveV5-C-Stage2.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/AltWaveV5-C-instruct/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/AltWaveV5-C-Stage2.pth
-rw-r--r-- 1 root root 5.7G Aug  8 16:05 ../model/AltWaveV5-C-Stage2.pth


In [11]:
# Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && python3 dragon_test.py "../model/AltWaveV5-C-Stage2.pth" "cuda fp32"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
--- DRAGON PROMPT ---
In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese. They can't do so because they're indeed afraid of the ones living in the wilderness. For example, the wolves will likely never find the monsters in a single circle. The wolves were able to answer the question of how long the platypus and the swallows they ate. The cats did not understand why the fox got the name "watch" and the fox would look out like a tiger in the leaves of the wolf. The dog's prey then died in the forest. The cat may no longer call the snakes. The owner's animal would have done a hunt, and if it was still friendly, the fox would attack the fox in the game. The cacodua group called the Tiger in the forest. The fox wa

In [12]:
# # Lets do a quick memory test
# # (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
# !python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/AltWaveV5-C-Stage2.pth"